![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Uncorrelated Assets
Finding uncorrelated assets allows you to find a portfolio that will, theoretically, be more diversified and resilient to extreme market events. When combined with other indicators and data sources, this can be an important component in building an algorithm that limits drawdown and remains profitable in choppy markets.

The first step is to execute the cell containing our correlation function and then grab the historical data for the securities and use this to fetch returns data.

In [1]:
# This cell is the function we use to get correlation ranks
def GetUncorrelatedAssets(returns, num_assets):
    # Get correlation
    correlation = returns.corr()
    
    # Find assets with lowest mean correlation, scaled by STD
    selected = []
    for index, row in correlation.iteritems():
        corr_rank = row.abs().mean()/row.abs().std()
        selected.append((index, corr_rank))

    # Sort and take the top num_assets
    selected = sorted(selected, key = lambda x: x[1])[:num_assets]
    
    return selected

In [2]:
# Import custom functions
qb = QuantBook()

tickers = ["SQQQ", "TQQQ", "TVIX", "VIXY", "SPLV",
           "SVXY", "UVXY", "EEMV", "EFAV", "USMV"]
symbols = [qb.AddEquity(x, Resolution.Minute) for x in tickers]

# Fetch history
history = qb.History(qb.Securities.Keys, 150, Resolution.Hour)

# Get hourly returns
returns = history.unstack(level = 1).close.transpose().pct_change().dropna()

Then, we calculate the correlation of the returns, which gives us a correlation matrix. In the GetUncorrelatedAssets function, we figure out which symbols have the lowest overall correlation with the rest of the symbols as a whole -- we want to find the five assets with the lowest average absolute correlation so that we can trade them without fearing that any pair are too highly correlated.

In [3]:
# Get 5 assets with least overall correlation
selected = GetUncorrelatedAssets(returns, 5)
selected

[('VIXY UT076X30D0MD', 6.908714329573668),
 ('UVXY V0H08FY38ZFP', 6.976330405968959),
 ('SVXY V0H08FY38ZFP', 7.041772632696755),
 ('SPLV UWBCA8CFMB1H', 7.109323797061953),
 ('TVIX US1QJNA3OM05', 7.113355127827484)]